In [1]:
from curve import Fr as Field, ec_mul, G1Point as G1
from merlin.merlin_transcript import MerlinTranscript
from mle2 import MLEPolynomial
from utils import log_2, log_2_ceiling, is_power_of_two, prime_field_inv, Scalar
from kzg10_non_hiding2 import Commitment, KZG10_PCS
from curve import Fr as Field, G1Point as G1, G2Point as G2, ec_pairing_check
from unipoly2 import UniPolynomial, UniPolynomialWithFft
from typing import TypeVar, Union, Optional
from random import Random, randint

In [2]:
BN254_Fr = GF(Field.field_modulus); BN254_Fr

Finite Field of size 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [3]:
Rp.<X> = BN254_Fr[]
B_mono = [X^i for i in range(0, 1024)]
def poly(cof, B=B_mono):
    return reduce(lambda acc, e: acc + e, [BN254_Fr(int(a)) * b for a, b in zip(cof, B)])
    
def powers(u, k):
    return [u^i for i in range(0, 2**k)]
    
def eval(f, D):
    n = len(D)
    k = log_2_ceiling(n)
    return [poly(f, powers(D[i], k)) for i in range(0, n)]
F = Field


In [4]:
kzg_pcs = KZG10_PCS(G1, G2, Field, 20, debug=True)

In [5]:
def commit(f: MLEPolynomial):
    evals = f.evals
    f_cm = kzg_pcs.commit(UniPolynomial(evals))
    return f_cm

In [6]:
tr = MerlinTranscript(b"test-mercury-pcs")

In [7]:
f_mle = MLEPolynomial([F(int(1)), F(int(3)), F(int(2)), F(int(1)),
                    F(int(2)), F(int(-2)), F(int(1)), F(int(0)),
                    F(int(-1)), F(int(2)), F(int(3)), F(int(1)),
                    F(int(3)), F(int(1)), F(int(-2)), F(int(3))], 4)
f_cm = commit(f_mle)
us = [F(int(2)), F(int(-1)), F(int(1)), F(int(-2))]
v = f_mle.evaluate(us)
v


-13

In [8]:
k = f_mle.num_var
us_l = us[:k//2]
us_r = us[k//2:]
vec_eq_l = MLEPolynomial.eqs_over_hypercube(us_l)
vec_eq_r = MLEPolynomial.eqs_over_hypercube(us_r)
n_l = len(vec_eq_l) # column
n_r = len(vec_eq_r) # row
vec_eq_l, vec_eq_r


([-2, 4, 1, -2], [0, 3, 0, -2])

In [9]:
alpha = F(int(3))
alpha_exp_4 = F(int(3^4))

$$
\tilde{f}(X_0, X_1, X_2, X_3) = 
\begin{bmatrix}
1 & X_2 & X_3 & X_2X_3 \\
\end{bmatrix}
\begin{bmatrix}
c_0 & c_1 & c_2 & c_3 \\
c_4 & c_5 & c_6 & c_7 \\
c_8 & c_9 & c_{10} & c_{11} \\
c_{12} & c_{13} & c_{14} & c_{15}
\end{bmatrix}
\begin{bmatrix}
1 \\
X_0 \\
X_1 \\
X_0X_1 \\
\end{bmatrix}
$$

In [10]:
f_mle.evals

[1, 3, 2, 1, 2, -2, 1, 0, -1, 2, 3, 1, 3, 1, -2, 3]

In [11]:
[f_mle.evals[i:i+4] for i in range(0, len(f_mle.evals), 4)]


[[1, 3, 2, 1], [2, -2, 1, 0], [-1, 2, 3, 1], [3, 1, -2, 3]]

In [12]:
h_evals = []
for j in range(n_l):
    h_evals.append([f_mle.evals[j+i*n_l] for i in range(n_r)] )
h_evals


[[1, 2, -1, 3], [3, -2, 2, 1], [2, 1, 3, -2], [1, 0, 1, 3]]

In [13]:
f0_mle = MLEPolynomial(h_evals[0], 2)
f1_mle = MLEPolynomial(h_evals[1], 2)
f2_mle = MLEPolynomial(h_evals[2], 2)
f3_mle = MLEPolynomial(h_evals[3], 2)
f0_mle.evals, f1_mle.evals, f2_mle.evals, f3_mle.evals


([1, 2, -1, 3], [3, -2, 2, 1], [2, 1, 3, -2], [1, 0, 1, 3])

In [14]:
f0_poly = poly(f0_mle.evals)
f1_poly = poly(f1_mle.evals)
f2_poly = poly(f2_mle.evals)
f3_poly = poly(f3_mle.evals)


In [15]:
f0_uni = UniPolynomial(f0_mle.evals)
f1_uni = UniPolynomial(f1_mle.evals)
f2_uni = UniPolynomial(f2_mle.evals)
f3_uni = UniPolynomial(f3_mle.evals)
f_uni = UniPolynomial(f_mle.evals)
f_uni, f_uni.evaluate(F(int(3)))

(1 + 3x + 2x^2 + x^3 + 2x^4 + -2x^5 + x^6 + -1x^8 + 2x^9 + 3x^10 + x^11 + 3x^12 + x^13 + -2x^14 + 3x^15,
 37056988)

In [16]:
f0_uni.evaluate(F(int(3^4))) + F(int(3)) * f1_uni.evaluate(F(int(3^4))) + F(int(3^2)) * f2_uni.evaluate(F(int(3^4))) + F(int(3^3)) * f3_uni.evaluate(F(int(3^4)))

37056988

In [17]:
f_poly = (f0_poly(X^4) + X*f1_poly(X^4) + X^2*f2_poly(X^4) + X^3*f3_poly(X^4))
f_poly, f_poly(BN254_Fr(int(3)))


(3*X^15 + 21888242871839275222246405745257275088548364400416034343698204186575808495615*X^14 + X^13 + 3*X^12 + X^11 + 3*X^10 + 2*X^9 + 21888242871839275222246405745257275088548364400416034343698204186575808495616*X^8 + X^6 + 21888242871839275222246405745257275088548364400416034343698204186575808495615*X^5 + 2*X^4 + X^3 + 2*X^2 + 3*X + 1,
 37056988)

In [18]:
q_poly = f_poly // (X^4-3)
r_poly = f_poly % (X^4-3)
q_poly, r_poly


(3*X^11 + 21888242871839275222246405745257275088548364400416034343698204186575808495615*X^10 + X^9 + 3*X^8 + 10*X^7 + 21888242871839275222246405745257275088548364400416034343698204186575808495614*X^6 + 5*X^5 + 8*X^4 + 30*X^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495609*X^2 + 13*X + 26,
 91*X^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495595*X^2 + 42*X + 79)

In [19]:
q_poly(BN254_Fr(int(alpha))), r_poly(BN254_Fr(int(alpha)))

(475058, 2464)

In [20]:
# Check if f[0..3] are correct
x_uni = UniPolynomial([F(int(0)), F(int(1))])

f_alt_eval_at_alpha = f0_uni.evaluate(alpha_exp_4) \
    + (f1_uni.evaluate(alpha_exp_4)*alpha) \
    + (f2_uni.evaluate(alpha_exp_4)*alpha*alpha) \
    + (f3_uni.evaluate(alpha_exp_4) * alpha*alpha*alpha)
f_alt_eval_at_alpha

37056988

In [21]:
q0_uni, r0 = f0_uni.div_by_linear_divisor(alpha)
q1_uni, r1 = f1_uni.div_by_linear_divisor(alpha)
q2_uni, r2 = f2_uni.div_by_linear_divisor(alpha)
q3_uni, r3 = f3_uni.div_by_linear_divisor(alpha)
q0_uni, r0, q1_uni, r1, q2_uni, r2, q3_uni, r3


(26 + 8x + 3x^2,
 79,
 13 + 5x + x^2,
 42,
 -8 + -3x + -2x^2,
 -22,
 30 + 10x + 3x^2,
 91)

In [22]:
r0 == f0_uni.evaluate(alpha), r1 == f1_uni.evaluate(alpha), r2 == f2_uni.evaluate(alpha), r3 == f3_uni.evaluate(alpha)


(True, True, True, True)

In [23]:
# Construct q(X)
#
#   q(X) = q0(X^4) + X*q1(X^4) + X^2*q2(X^4) + X^3*q3(X^4)
#   
#  Or, the coefficients of q(X) are:
#
#   [q0[0], q1[0], q2[0], q3[0], 
#    q0[1], q1[1], q2[1], q3[1], 
#    q0[2], q1[2], q2[2], q3[2]]
#
#  Please note that deg(q) < 12

q_coeffs = []
for i in range(len(q0_uni.coeffs)):
    q_coeffs.append(q0_uni.coeffs[i])
    q_coeffs.append(q1_uni.coeffs[i])
    q_coeffs.append(q2_uni.coeffs[i])
    q_coeffs.append(q3_uni.coeffs[i])
q_uni = UniPolynomial(q_coeffs)
q_uni.evaluate(alpha), q_coeffs

(475058, [26, 13, -8, 30, 8, 5, -3, 10, 3, 1, -2, 3])

In [24]:
def compute_quotient(coeffs, k):
    num_col = 2**k
    num_row = len(coeffs) // num_col
    print(f"num_col: {num_col}, num_row: {num_row}")

    q_coeffs = [F.zero()] * (num_col * (num_row-1))
    for j in range(num_col):
        fi = coeffs[j::num_row]

        qi, ri = UniPolynomial(fi).div_by_linear_divisor(alpha)
        for i in range(num_row-1):
            q_coeffs[j+i*num_row] = qi.coeffs[i]
    return q_coeffs



In [25]:
compute_quotient(f_uni.coeffs, 2)

num_col: 4, num_row: 4


[26, 13, -8, 30, 8, 5, -3, 10, 3, 1, -2, 3]

In [26]:
q_eval_at_alpha = q0_uni.evaluate(alpha_exp_4) \
    + (q1_uni.evaluate(alpha_exp_4)*alpha) \
    + (q2_uni.evaluate(alpha_exp_4)*alpha*alpha) \
    + (q3_uni.evaluate(alpha_exp_4) * alpha*alpha*alpha)
q_eval_at_alpha

475058

In [27]:
def compute_product_poly(a: list[Field], b: list[Field]):
    assert len(a) == len(b), "Length of a and b must be the same"
    l = len(a)
    s = [F.zero()] * (l-1)
    dot_product = F.zero()
    for i in range(l):
        for j in range(l):
            if abs(i-j) == 0:
                dot_product += a[i] * b[j]
            else:
                s[abs(i-j)-1] += a[i] * b[j]
    return s, dot_product

In [28]:
def inner_product(a, b, z):
    return sum([a[i] * b[i] for i in range(len(a))], z)

In [29]:
a = [F(int(1)), F(int(2)), F(int(3)), F(int(4))]
b = [F(int(4)), F(int(2)), F(int(3)), F(int(1))]
a_uni = UniPolynomial(a)
b_uni = UniPolynomial(b)
s, v = compute_product_poly(a, b)
s_uni = UniPolynomial(s)
s, v, v == inner_product(a, b, F(int(0)))

([37, 25, 17], 21, True)

In [30]:
# Check if s(X) is correct
#
#    a(X)b(1/X) + a(1/X)b(X) = 2v + X*s(X) + (1/X)*s(1/X)
#
z = F.rand()
z_inv = int(1)/z
# a_uni.evaluate(z) * b_uni.evaluate(z_inv), F(int(2)) * v
a_uni.evaluate(z) * b_uni.evaluate(z_inv) + a_uni.evaluate(z_inv) * b_uni.evaluate(z) \
    == F(int(2))*v + z*s_uni.evaluate(z) + z_inv*s_uni.evaluate(z_inv)

True

In [31]:
q_coeffs = []
for i in range(len(q0_uni.coeffs)):
    q_coeffs.append(q0_uni.coeffs[i])
    q_coeffs.append(q1_uni.coeffs[i])
    q_coeffs.append(q2_uni.coeffs[i])
    q_coeffs.append(q3_uni.coeffs[i])
q_coeffs

[26, 13, -8, 30, 8, 5, -3, 10, 3, 1, -2, 3]

In [32]:
q, r = UniPolynomial.polynomial_division_with_remainder(f_uni.coeffs, 
                        [-alpha, F(int(0)), F(int(0)), F(int(0)), F(int(1))])
q, r

([26, 13, -8, 30, 8, 5, -3, 10, 3, 1, -2, 3], [79, 42, -22, 91])

In [33]:
r_uni = UniPolynomial(r)

In [34]:
r_uni.evaluate(F(int(3)))

2464

In [35]:
f0_uni + f1_uni, Scalar(F(int(3))) * f0_uni


(4 + x^2 + 4x^3, 3 + 6x + -3x^2 + 9x^3)

In [36]:
h = Scalar(vec_eq_l[0]) * f0_uni + Scalar(vec_eq_l[1]) * f1_uni  \
    + Scalar(vec_eq_l[2]) * f2_uni + Scalar(vec_eq_l[3]) * f3_uni
h

10 + -11x + 11x^2 + -10x^3

In [37]:
f0_mle, -f0_mle

(MLEPolynomial([1, 2, -1, 3], 2), MLEPolynomial([-1, -2, 1, -3], 2))

In [38]:
f0_mle - f1_mle, f0_mle, f1_mle

(MLEPolynomial([-2, 4, -3, 2], 2),
 MLEPolynomial([1, 2, -1, 3], 2),
 MLEPolynomial([3, -2, 2, 1], 2))

In [39]:
v = f_mle.evaluate(us)
v

-13

## 2. Prove evaluation

In [40]:
debug = 1
transcript = MerlinTranscript(b"test-mercury-pcs")
transcript.absorb(b"commitment", f_cm.cm)
transcript.absorb(b"point", us)
transcript.absorb(b"value", v)


### Step 1

1. compute $h(X)$
2. commit $h(X)$

In [41]:

k = f_mle.num_var
us_l = us[:k//2]
us_r = us[k//2:]
vec_eq_l = MLEPolynomial.eqs_over_hypercube(us_l)
vec_eq_r = MLEPolynomial.eqs_over_hypercube(us_r)
n_l = len(vec_eq_l) # column
n_r = len(vec_eq_r) # row
vec_eq_l, vec_eq_r

([-2, 4, 1, -2], [0, 3, 0, -2])

In [42]:
# Compute h(X)
h_coeffs = []
for i in range(n_r):
    h_coeffs.append(sum([f_mle.evals[i*n_l+j] * vec_eq_l[j] for j in range(n_r)], F(int(0))))
h_uni = UniPolynomial(h_coeffs)
h_cm = kzg_pcs.commit(h_uni)
transcript.absorb(b"h_cm", h_cm.cm)
h_cm

Commitment((341203988594841721703751627390561602518991312853592249390238506514023511558, 4570164882443154697207366432674390476737733102807743213900207502309583215167))

In [43]:
if debug > 0:
    print(f"P> check: folded polynomial h(X)")
    coeffs = f_mle.evals
    eqs = MLEPolynomial.eqs_over_hypercube(us)
    lhs = inner_product(h_coeffs, vec_eq_r, F(int(0))) 
    rhs = inner_product(coeffs, eqs, F(int(0)))
    assert lhs == rhs, f"lhs: {lhs}, rhs: {rhs}"
    print(f"P> check: folded polynomial h(X) passed")


P> check: folded polynomial h(X)
P> check: folded polynomial h(X) passed


### Step 2

- (a). get $\alpha\in\mathbb{F}_p$ from the verifier
- (b). compute $q(X)$ and $g(X)$, s.t.

$$
f(X) = (X^b - \alpha) * q(X) + g(X)
$$

In [44]:
# Step 2

alpha = transcript.squeeze(Field, b"alpha", 4)
alpha

66954866

In [45]:
def div_by_quadratic_binomial(coeffs, alpha, k):
    num_col = 2**k
    num_row = len(coeffs) // num_col
    print(f"num_col: {num_col}, num_row: {num_row}")

    q_coeffs = [F.zero()] * (num_col * (num_row-1))
    r_coeffs = [F.zero()] * num_col
    for j in range(num_col):
        fi = coeffs[j::num_row]
        qi, ri = UniPolynomial(fi).div_by_linear_divisor(alpha)
        for i in range(num_row-1):
            q_coeffs[j+i*num_row] = qi.coeffs[i]
        r_coeffs[j] = ri
    return q_coeffs, r_coeffs

In [46]:
# Compute q(X) and g(X)

q_coeffs, g_coeffs = div_by_quadratic_binomial(f_mle.evals, alpha, 2)
q_uni = UniPolynomial(q_coeffs)
g_uni = UniPolynomial(g_coeffs)
q_cm = kzg_pcs.commit(q_uni)
g_cm = kzg_pcs.commit(g_uni)
q_cm, g_cm

num_col: 4, num_row: 4


(Commitment((6873374277395425445538081471403900421153044372429899318315463318322988015074, 16690292592350444777928793433203390570537489251096203454692664097972278279727)),
 Commitment((12744849022295298739892606625958984031642490183242839912364573373366770170425, 10807392124513062104048213661706791445849571919278941606978474073386634973494)))

In [47]:
if debug > 0:
    print(f"P> check: quotient polynomial q(X) and remainder polynomial g(X)")
    q, r = UniPolynomial.polynomial_division_with_remainder(f_mle.evals, 
                            [-alpha, F(int(0)),  F(int(0)), F(int(0)), F(int(1))])
    assert q == q_coeffs, f"q: {q}, q_coeffs: {q_coeffs}"
    print(f"P> check: quotient polynomial q(X) passed")
    assert r == g_coeffs, f"r: {r}, r_coeffs: {g_coeffs}"
    print(f"P> check: remainder polynomial g(X) passed")

P> check: quotient polynomial q(X) and remainder polynomial g(X)
P> check: quotient polynomial q(X) passed
P> check: remainder polynomial g(X) passed


In [48]:
# Commit and send q(X) and g(X)

transcript.absorb(b"q_cm", q_cm.cm)
transcript.absorb(b"g_cm", g_cm.cm)

### Step 3

- (a). Get $\gamma\in\mathbb{F}_p$ from the verifier
- (b). Compute $s(X)$ and send its commitment $C_s$
- (c). Compute $d(X)$ and send its commitment $C_d$



In [75]:
gamma = transcript.squeeze(Field, b"gamma", 4)
gamma, type(gamma)

(89040678, <class 'curve.Fr'>)

In [50]:
def compute_product_poly(a: list[Field], b: list[Field]):
    assert len(a) == len(b), "Length of a and b must be the same"
    l = len(a)
    s = [F.zero()] * (l-1)
    dot_product = F.zero()
    for i in range(l):
        for j in range(l):
            if abs(i-j) == 0:
                dot_product += a[i] * b[j]
            else:
                s[abs(i-j)-1] += a[i] * b[j]
    return s, dot_product

In [76]:
# Compute s1(X) and s2(X), and send [s(X)]
#
#  where s(X) = s1(X) + gamma * s2(X)

s1_coeffs, v1 = compute_product_poly(g_coeffs, vec_eq_l)
s2_coeffs, v2 = compute_product_poly(h_coeffs, vec_eq_r)
s1_uni = UniPolynomial(s1_coeffs)
s2_uni = UniPolynomial(s2_coeffs)
s_uni = s1_uni + Scalar(gamma) * s2_uni
s_cm = kzg_pcs.commit(s_uni)
s_cm

Commitment((196218153828482921355178233114718791152912307737878072738063461265417166874, 394014398374792061255415805359155323151193649000371028071552966415569233135))

In [77]:
if debug > 0:
    print(f"P> check: s1(X) and s2(X)")
    assert v2 == v, f"v2: {v2}, v: {v}"
    assert v1 == h_uni.evaluate(alpha), f"v1: {v1}, h_uni.evaluate(alpha): {h_uni.evaluate(alpha)}"
    z = F.rand()
    z_inv = int(1)/z
    g_uni = UniPolynomial(g_coeffs)
    p1_uni = UniPolynomial(vec_eq_l)
    h_uni = UniPolynomial(h_coeffs)
    p2_uni = UniPolynomial(vec_eq_r)
    s1_uni = UniPolynomial(s1_coeffs)
    s2_uni = UniPolynomial(s2_coeffs)
    assert g_uni.evaluate(z) * p1_uni.evaluate(z_inv) + g_uni.evaluate(z_inv) * p1_uni.evaluate(z) \
        == F(int(2))*v1 + z*s1_uni.evaluate(z) + z_inv*s1_uni.evaluate(z_inv)
    print(f"P> check: s1(X) passed")
    assert h_uni.evaluate(z) * p2_uni.evaluate(z_inv) + h_uni.evaluate(z_inv) * p2_uni.evaluate(z) \
        == F(int(2))*v2 + z*s2_uni.evaluate(z) + z_inv*s2_uni.evaluate(z_inv)
    print(f"P> check: s2(X) passed")



P> check: s1(X) and s2(X)
P> check: s1(X) passed
P> check: s2(X) passed


In [78]:
# Compute d(X) where d(X) = X^{l-1} * g(1/X)

d_coeffs = g_coeffs[::-1]
d_uni = UniPolynomial(d_coeffs)
d_cm = kzg_pcs.commit(d_uni)
d_cm

Commitment((411548608121372520815472318385633818999017454145389265932878881921792672285, 11683702693697689722457230903516371286641048263922663281742983635724242819079))

In [79]:
if debug > 0:
    print(f"P> check: d(X)")
    z = F.rand()
    z_inv = int(1)/z
    d_uni = UniPolynomial(d_coeffs)
    g_uni = UniPolynomial(g_coeffs)
    lhs = d_uni.evaluate(z)
    assert lhs == z^(int(n_l)-int(1)) * g_uni.evaluate(z_inv)
    print(f"P> check: d(X) passed")

P> check: d(X)
P> check: d(X) passed


In [80]:
transcript.absorb(b"s_cm", s_cm.cm)
transcript.absorb(b"d_cm", d_cm.cm)

### Step 4

- (a). Get $\zeta\in\mathbb{F}_p$ from the verifier
- (b). Compute and send $g(\zeta), g(1/\zeta), h(\zeta), h(1/\zeta), s(\zeta), s(1/\zeta)$ and send them to the verifier
- (c). Compute the quotient polynomial $t(X)$ and send its commitment $C_t$ 

$$
t(X) = \frac{f(X) - q(X)(z^b - \alpha) - g(\zeta)}{X - \zeta}
$$


In [81]:
zeta = transcript.squeeze(Field, b"zeta", 4)
zeta_inv = zeta.inv()

In [82]:
g_at_zeta = g_uni.evaluate(zeta)
g_at_zeta_inv = g_uni.evaluate(zeta_inv)
h_at_zeta = h_uni.evaluate(zeta)
h_at_zeta_inv = h_uni.evaluate(zeta_inv)
s_at_zeta = s_uni.evaluate(zeta)
s_at_zeta_inv = s_uni.evaluate(zeta_inv)

In [88]:
u = f_uni - q_uni * Scalar(zeta^n_l - alpha) - Scalar(g_at_zeta)
u

-320010484181115610812123662559302472706056545349919408 + -103508575654681906348588614540275275117858512989098647x + 207017140487745510874267438536765553073217321966678577x^2 + -310525719234318449659817744109557372277347152542366849x^3 + -4637836421663923373260987705813334791825921673x^4 + -1545945527763164405815148009344596766480018702x^5 + 3091890893900758967445839696468738025345902976x^6 + -4637836467842657614456546655019179222572817225x^7 + -69268101361793338423808766646120343326x^8 + -23089367120597779474602922215373447773x^9 + 46178734241195558949205844430746895553x^10 + -69268101361793338423808766646120343324x^11 + 3x^12 + x^13 + -2x^14 + 3x^15

In [89]:
t_uni, t_eval = u.div_by_linear_divisor(zeta)
t_cm = kzg_pcs.commit(t_uni)
t_cm, t_eval


(Commitment((7564077621549655343295293161727763898648355577163371329825382307061906824629, 20961728419572618406325587578344293151556639381818099483799713097095237121275)),
 0)

In [90]:
if debug > 0:
    print(f"P> check: quotient polynomial t(X)")
    assert t_eval == Field.zero(), f"t_eval: {t_eval}"
    z = F.rand()
    t_z = t_uni.evaluate(z)
    f_z = f_uni.evaluate(z)
    q_z = q_uni.evaluate(z)
    assert t_z * (z - zeta) == f_z - q_z * (zeta^n_l - alpha) - g_at_zeta
    print(f"P> check: quotient polynomial t(X) passed")


P> check: quotient polynomial t(X)
P> check: quotient polynomial t(X) passed


In [92]:
if debug > 0:
    print(f"P> check: C_t")
    g_G1 = kzg_pcs.params['g']
    h_G2 = kzg_pcs.params['h']
    h_tau_G2 = kzg_pcs.params['tau_h']
    lhs1 = f_cm.cm + t_cm.cm.ec_mul(zeta) - q_cm.cm.ec_mul(zeta^n_l - alpha) - g_G1.ec_mul(g_at_zeta)
    lhs2 = h_G2
    rhs1 = t_cm.cm
    rhs2 = h_tau_G2
    print(f"type(lhs1)= {type(lhs1)}, type(lhs2)= {type(lhs2)}, type(rhs1)= {type(rhs1)}, type(rhs2)= {type(rhs2)}")
    checked = ec_pairing_check([lhs1, rhs1], [-lhs2, rhs2])
    assert checked, "C_t is not valid"
    print(f"P> check: C_t passed")


P> check: C_t
type(lhs1)= <class 'curve.G1Point'>, type(lhs2)= <class 'curve.G2Point'>, type(rhs1)= <class 'curve.G1Point'>, type(rhs2)= <class 'curve.G2Point'>
P> check: C_t passed


In [93]:
transcript.absorb(b"t_cm", t_cm.cm)
transcript.absorb(b"g_at_zeta", g_at_zeta)
transcript.absorb(b"g_at_zeta_inv", g_at_zeta_inv)
transcript.absorb(b"h_at_zeta", h_at_zeta)
transcript.absorb(b"h_at_zeta_inv", h_at_zeta_inv)
transcript.absorb(b"s_at_zeta", s_at_zeta)
transcript.absorb(b"s_at_zeta_inv", s_at_zeta_inv)

### Step 5

Aggregate the evaluation proofs

In [94]:
# Aggregate the evaluation proofs

eta = transcript.squeeze(Field, b"eta", 4)

a_uni = g_uni + Scalar(eta) * h_uni + Scalar(eta*eta) * s_uni
a_at_zeta_inv, a_at_zeta_inv_arg = kzg_pcs.prove_evaluation(a_uni, zeta_inv)

a_uni += Scalar(eta*eta*eta) * d_uni
a_at_zeta, a_at_zeta_arg = kzg_pcs.prove_evaluation(a_uni, zeta)

h_uni_at_alpha, h_at_alpha_arg = kzg_pcs.prove_evaluation(h_uni, alpha)
a_at_zeta_arg, a_at_zeta_inv_arg, h_at_alpha_arg



({'w_cm': Commitment((15522446906494288407282696648261850622287101021721313436696028069912532784720, 603214464022950610669451026422499576356824841616091783390441425826994905145))},
 {'w_cm': Commitment((21807297083416410354924359503853066118819031366858090987581267815333402259120, 3665392969967324964834023121604975099446719084408716190589686241189390173360))},
 {'w_cm': Commitment((19277402036953961621331476720166919837657448960632489771751307109877544454558, 14785722924673158890380759949870394812979469962011416906141661546783155852289))})

In [96]:
if debug > 0:
    print(f"P> check: h(alpha)")
    p1_uni = UniPolynomial(vec_eq_l)
    p2_uni = UniPolynomial(vec_eq_r)
    p1_at_zeta = p1_uni.evaluate(zeta)
    p2_at_zeta = p2_uni.evaluate(zeta)
    p1_at_zeta_inv = p1_uni.evaluate(zeta_inv)
    p2_at_zeta_inv = p2_uni.evaluate(zeta_inv)
    h_alpha = (h_at_zeta * p2_at_zeta_inv) + (h_at_zeta_inv * p2_at_zeta) - v - v
    h_alpha *= gamma 
    h_alpha += (g_at_zeta * p1_at_zeta_inv) + (g_at_zeta_inv * p1_at_zeta)
    h_alpha -= zeta * s_at_zeta + zeta_inv * s_at_zeta_inv

    d_at_zeta = zeta^(int(n_l-1)) * g_at_zeta_inv
    assert d_at_zeta == d_uni.evaluate(zeta), f"d_at_zeta: {d_at_zeta}, d_uni.evaluate(zeta): {d_uni.evaluate(zeta)}"
    assert h_uni_at_alpha * Field(int(2)) == h_alpha, f"h_uni_at_alpha: {h_uni_at_alpha}, h_alpha: {h_alpha}"
    print(f"P> check: h(alpha) passed")


P> check: h(alpha)
P> check: h(alpha) passed


In [98]:
if debug > 0:
    print(f"P> check: aggregate evaluation proofs")
    C_a = g_cm.cm + h_cm.cm.ec_mul(eta) + s_cm.cm.ec_mul(eta*eta) 
    checked1 = kzg_pcs.verify_evaluation(Commitment(C_a), zeta_inv, a_at_zeta_inv, a_at_zeta_inv_arg)
    C_a += d_cm.cm.ec_mul(eta*eta*eta)
    checked2 = kzg_pcs.verify_evaluation(Commitment(C_a), zeta, a_at_zeta, a_at_zeta_arg)
    assert checked1 and checked2, f"checked1: {checked1}, checked2: {checked2}"
    checked3 = kzg_pcs.verify_evaluation(h_cm, alpha, h_uni_at_alpha, h_at_alpha_arg)
    assert checked3, f"checked3: {checked3}"
    print(f"P> check: aggregate evaluation proofs passed")

P> check: aggregate evaluation proofs
P> check: aggregate evaluation proofs passed


In [101]:
ff = UniPolynomial([F(int(2)), F(int(6)), F(int(10)), F(int(14))])
ff.evaluate(F(int(3194597718)))


456432506869271084770462484798